# Import Neccessary Librarys

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from keras.utils import pad_sequences, to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional
from keras import Sequential
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load Dataset

In [42]:
df = pd.read_csv(r"F:\Projects'\Next-Word-Prediction-main\Next-Word-Prediction-main\Dataset\medium_data.csv")

In [43]:
df.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6508 entries, 0 to 6507
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            6508 non-null   int64 
 1   url           6508 non-null   object
 2   title         6508 non-null   object
 3   subtitle      3479 non-null   object
 4   image         6361 non-null   object
 5   claps         6508 non-null   int64 
 6   responses     6508 non-null   object
 7   reading_time  6508 non-null   int64 
 8   publication   6508 non-null   object
 9   date          6508 non-null   object
dtypes: int64(3), object(7)
memory usage: 508.6+ KB


In [45]:
df.isnull().sum()

id                 0
url                0
title              0
subtitle        3029
image            147
claps              0
responses          0
reading_time       0
publication        0
date               0
dtype: int64

# Use title and subtitle column

In [46]:
df = df[["title", "subtitle"]]

In [48]:
df.head()

0    A Beginner’s Guide to Word Embedding with Gens...
1    Hands-on Graph Neural Networks with PyTorch & ...
2                         How to Use ggplot2 in Python
3    Databricks: How to Save Files in CSV on Your L...
4    A Step-by-Step Implementation of Gradient Desc...
dtype: object

In [47]:
df = pd.concat([df["title"], df["subtitle"]], ignore_index=True)
df.head()

0    A Beginner’s Guide to Word Embedding with Gens...
1    Hands-on Graph Neural Networks with PyTorch & ...
2                         How to Use ggplot2 in Python
3    Databricks: How to Save Files in CSV on Your L...
4    A Step-by-Step Implementation of Gradient Desc...
dtype: object

# Preprocess the Dataset

In [49]:
df.isnull().sum()

np.int64(3029)

In [ ]:
df.dropna(inplace=True)

In [50]:
df.isnull().sum()

np.int64(3029)

In [51]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional, for better lemmatization


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [52]:
from nltk.tokenize import word_tokenize

In [53]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# One-time NLTK downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean(text):
    words = word_tokenize(text.lower())
    words = [
        lemmatizer.lemmatize(word, pos="v")
        for word in words
        if word.isalpha() and word not in stop_words and len(word) > 1
    ]
    return ' '.join(words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import nltk
nltk.data.path
import nltk
nltk.download('punkt')


In [54]:
df = df.apply(lambda x: clean(x))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\athun/nltk_data'
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\share\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\lib\\nltk_data"
    - 'C:\\Users\\athun\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [55]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required data (only once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean(text):
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(w, pos="v") for w in words if w.isalpha() and w not in stop_words]
    return ' '.join(words)

# Sample DataFrame
df = pd.DataFrame({
    'text': ["The boys were running quickly!", "Dogs bark loudly.", "He is eating apples."]
})

# Apply only on the text column
df['cleaned'] = df['text'].apply(clean)

print(df)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\athun/nltk_data'
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\share\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\lib\\nltk_data"
    - 'C:\\Users\\athun\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [56]:
import nltk

nltk.download('punkt')         # sentence tokenizer
nltk.download('stopwords')     # for stop word removal
nltk.download('wordnet')       # for lemmatization
nltk.download('omw-1.4')       # lemmatizer vocab


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [57]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean(text):
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(w, pos='v') for w in words if w.isalpha() and w not in stop_words]
    return ' '.join(words)


In [58]:
df['cleaned'] = df['text'].apply(clean)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\athun/nltk_data'
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\share\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\lib\\nltk_data"
    - 'C:\\Users\\athun\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [59]:
from nltk.tokenize import word_tokenize
text = "Hello, this is a test sentence!"
print(word_tokenize(text))


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\athun/nltk_data'
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\share\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\lib\\nltk_data"
    - 'C:\\Users\\athun\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [62]:
import nltk

# OPTIONAL but good to ensure a clean path
nltk.data.path.append("C:/Users/athun/nltk_data")

# NOW download punkt
nltk.download("punkt")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\athun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
from nltk.tokenize import word_tokenize
print(word_tokenize("This is a test."))


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\athun/nltk_data'
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\share\\nltk_data"
    - "f:\\Projects'\\Next-Word-Prediction-main\\.venv\\lib\\nltk_data"
    - 'C:\\Users\\athun\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/Users/athun/nltk_data'
**********************************************************************


In [ ]:
df.head()

# max_len and vocab_size

In [ ]:
max_len = max(len(line) for line in df)
max_len

In [ ]:
tokenizer = Tokenizer(oov_token="<nan>")
tokenizer.fit_on_texts(df)
tokenizer.word_index

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

# Data Generator Function

In [ ]:
def data_generator(df, tokenizer, max_len, vocab_size, batch_size):
  X, y = list(), list()
  n = 0
  while True:
    for line in df:
      n+=1
      seq = tokenizer.texts_to_sequences([line])[0]
      for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], padding="post", maxlen=max_len)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

        X.append(in_seq)
        y.append(out_seq)

      if n == batch_size:
        yield np.array(X), np.array(y)
        X, y = list(), list()
        n = 0

# Model Building

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100),
    Bidirectional(LSTM(150)),
    Dense(vocab_size, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
from keras.models import load_model
model = load_model("/content/drive/MyDrive/Project/Next Word Prediction/_model_.h5")

In [ ]:
epochs = 10
batch_size = 32
steps_per_epoch = len(df) // batch_size
history = []

for i in range(1, epochs+1):
    print(f"Epoch {i}")
    data = data_generator(df, tokenizer, max_len, vocab_size, batch_size)
    hist = model.fit(data, steps_per_epoch=steps_per_epoch, epochs=1)
    history.append(hist)

# Save

In [ ]:
model.save("/content/drive/MyDrive/Project/Next Word Prediction/_model_.h5")

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/Next Word Prediction/tokenizer.pkl", "wb") as f:
  pickle.dump(tokenizer, f)
with open("/content/drive/MyDrive/Colab Notebooks/Next Word Prediction/History.pkl", "wb") as f:
  pickle.dump(history, f)

# Loss vs Accuracy

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/Next Word Prediction/History.pkl", "rb") as f:
  history = pickle.load(f)

In [ ]:
loss = []
accuracy = []
for hist in history:
  loss.append(hist.history['loss'])
  accuracy.append(hist.history['accuracy'])

In [ ]:
plt.plot([i for i in range(len(loss))], loss, label="loss"),
plt.title("Loss"),
plt.xlabel("Epoch"),
plt.ylabel("loss")
plt.grid(),
plt.legend(),
plt.show()

In [ ]:
plt.plot([i for i in range(len(accuracy))], accuracy, color="orange", label="accuracy"),
plt.title("Accuracy"),
plt.xlabel("Epoch"),
plt.ylabel("accuracy")
plt.grid(),
plt.legend(),
plt.show()

In [ ]:
plt.plot([i for i in range(len(loss))], loss, label="loss"),
plt.plot([i for i in range(len(accuracy))], accuracy, color="orange", label="accuracy"),
plt.title("Loss Vs Accuracy"),
plt.xlabel("Epoch"),
plt.ylabel("loss/accuracy")
plt.grid(),
plt.legend(),
plt.show()

# Prediction

In [ ]:
def integer_to_text(tokenizer, label):
  for idx, val in tokenizer.word_index.items():
    if val == label:
      return idx
  return None

In [ ]:
import numpy as np
from keras.models import load_model

def load_tokenizer_model():
  model = load_model("/content/drive/MyDrive/Project/Next Word Prediction/model.h5")
  with open("/content/drive/MyDrive/Project/Next Word Prediction/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
  max_len = 340

  return model, tokenizer, max_len

def generate_next_word(text, next_words):
    for _ in range(next_words):
        seq = tokenizer.texts_to_sequences([text])[0]
        seq = pad_sequences([seq], padding="post", maxlen=max_len)[0]
        # Reshape the sequence to add a batch dimension
        seq = np.reshape(seq, (1, len(seq), 1))  # Assuming feature_dim is 1
        label_probabilities = model.predict(seq, verbose=False).flatten()

        # Get the indices of the top 3 probabilities
        top_3_indices = np.argsort(label_probabilities)[-3:][::-1]

        # Randomly select one of the top 3 indices
        selected_index = np.random.choice(top_3_indices)

        # selected_index = np.argmax(label_probabilities)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == selected_index:
                output_word = word
                break
        output_word = output_word.replace("…", "")
        text += " " + output_word
    return text

model, tokenizer, max_len = load_tokenizer_model()

In [ ]:
generate_next_word("How to build your", 20)